In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
matrix = pd.read_table('data/newsgroups5/matrix.txt',delimiter=',',header=None)
terms = pd.read_table('data/newsgroups5/terms.txt',delimiter=',',header=None)

classes = pd.read_table('data/newsgroups5/classes.txt',delimiter=',',header=None)


In [3]:
print('\n matrix ',matrix.shape,'\n terms ',terms.shape,'\n classes ',classes.shape)


 matrix  (9328, 2500) 
 terms  (9328, 1) 
 classes  (2501, 5)


In [4]:
cat_name_classes= {0:'windows',1:'crypt',2:'christian', 3:'hockey',4:'forsale'}
print(cat_name_classes.values())

dict_values(['windows', 'crypt', 'christian', 'hockey', 'forsale'])


In [5]:
classes.head()

,0,1,2,3,4
0,% Five Categories: windows (0),crypt (1),christian (2),hockey (3),forsale (4)
1,0 0,NaN,NaN,NaN,NaN
2,1 1,NaN,NaN,NaN,NaN
3,2 1,NaN,NaN,NaN,NaN
4,3 1,NaN,NaN,NaN,NaN


In [6]:
dataMat = np.mat(matrix.T)
dataMat.shape

(2500, 9328)

### a. Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.

In [7]:
def calculateEuclideanDistance(x,D):
    dist = np.sqrt((D -x)**2).sum(axis=1)
    return dist

def calculateCosineSimilarityDistance(x,D):
    D_norm = np.array([np.linalg.norm(D[i]) for i in range(len(D))])
    x_norm = np.linalg.norm(x)
    sims = np.dot(D,x)/(D_norm * x_norm)
    # The distance measure will be the inverse of Cosine similarity
    dist = 1 - sims
    
    return dist



### b. Load the data set [Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [109]:
matrix = pd.read_table('data/newsgroups5/matrix.txt',delimiter=',',header=None).T
terms = pd.read_table('data/newsgroups5/terms.txt',delimiter=',',header=None)
classes = np.genfromtxt('data/newsgroups5/classes.txt',delimiter=' ', dtype=int, skip_header=1, usecols=(1))


In [110]:
print('\n matrix ',matrix.shape,'\n terms ',terms.shape,'\n classes ',classes.shape)


 matrix  (2500, 9328) 
 terms  (9328, 1) 
 classes  (2500,)


In [111]:
matrix.describe()

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.00000,2500.000000,2500.000000,2500.000000,2500.000000,...,2500.000000,2500.000000,2500.000000,2500.000000,2500.00000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,0.006400,0.002400,0.032800,0.004800,0.012000,0.00400,0.022400,0.001600,0.031200,0.084800,...,0.005200,0.006400,0.001600,0.007600,0.00280,0.003200,0.020000,0.004400,0.002400,0.006000
std,0.138444,0.048941,0.954195,0.084734,0.227765,0.06918,0.248843,0.048973,0.244644,0.345336,...,0.103814,0.187545,0.048973,0.194827,0.08715,0.069222,0.523555,0.166107,0.084836,0.156121
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6.000000,1.000000,47.000000,2.000000,9.000000,2.00000,7.000000,2.000000,6.000000,4.000000,...,3.000000,7.000000,2.000000,8.000000,3.00000,2.000000,20.000000,8.000000,4.000000,5.000000


In [112]:
 matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Columns: 9328 entries, 0 to 9327
dtypes: int64(9328)
memory usage: 177.9 MB


In [17]:
vs_matrix = np.mat(matrix)
vs_matrix.shape

(2500, 9328)

In [115]:
from sklearn.model_selection import train_test_split
vs_train, vs_test, vs_target_train, vs_target_test = train_test_split(vs_matrix, classes, test_size=0.2, random_state=33)


In [157]:
print("train : ",vs_train.shape)
print("Test : ",vs_test.shape)

train :  (2000, 9328)
Test :  (500, 9328)


#### 2.c Perform Kmeans clustering on the training data

In [117]:
# def distEuclid(vecA, vecB):
#     return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

## get random centroids
def randCent(dataSet, k):
    n = np.shape(dataSet)[1]
    centroids = np.zeros((k,n), dtype=float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * np.random.rand(k)
    return centroids

In [118]:
print('max = ',max(dataMat[:,2499]),'min = ',min(dataMat[:,2499]))

max =  [[7]] min =  [[0]]


In [24]:
%%time
centroids = randCent(dataMat,2)

Wall time: 4min 15s


In [119]:
centroids

array([[0.        , 0.00990099, 0.00247525, ..., 0.        , 0.        ,
        0.        ],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.00440252, 0.        , 0.04528302, ..., 0.00125786, 0.00062893,
        0.00377358],
       [0.        , 0.        , 0.        , ..., 0.16666667, 0.        ,
        0.        ],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [120]:
def kmeans(D,k,distMeas=calculateCosineSimilarityDistance,createCentroid = randCent):
    m = np.shape(D)[0]
    
    ## cluster assignment matrix with 2 cols. one for cluster index and 2nd one for cluster error (distance from cluster centroid to the current point)
    clusterAssment = np.mat(np.zeros((m,2)))
    
    print('calculating centroids : Start')
    centroids = createCentroid(D,k)
    print('calculating centroids : End','\n centroids are :: \n',centroids)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        ## loop over data points and assign assign it to the closest centroid
        for i in range(m):
            minDIst = np.inf;
            minIndex = -1;
            for j in range(k):
                distJI = distMeas(centroids[j,:],D[i,:])
                ##print('Distance ',distJI)
                if(distJI < minDIst):
                    minDIst = distJI;
                    minIndex = j;
            if clusterAssment[i,0] !=minIndex : clusterChanged = True
            clusterAssment[i,:] = minIndex, minDIst**2
        #print(" centroids : ",centroids)
        
        ## update the centroid
        for cent in range(k):
            ptsInClust = D[np.nonzero(clusterAssment[:,0] == cent)[0]]
            centroids[cent,:] = np.mean(ptsInClust,axis=0)
    return centroids, clusterAssment

In [121]:
np.shape(dataMat)[0]

2500

In [122]:
np.zeros((9328,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [123]:
%%time
centroids, clusterAssment = kmeans(vs_train,5)

calculating centroids : Start
calculating centroids : End 
 centroids are :: 
 [[ 0.87576481  0.83924234 17.25375415 ...  1.9828995   0.99346734
   3.9044213 ]
 [ 0.43286733  0.71050652 22.16624865 ...  0.49760771  0.67370249
   1.38507032]
 [ 0.35554373  0.446494   14.12931283 ...  0.05726546  0.28993401
   2.06636939]
 [ 0.84502585  0.36845934 46.44965933 ...  1.58061292  0.18807721
   0.99625068]
 [ 0.92128585  0.42422509 27.5323093  ...  1.00610969  0.20254243
   1.701208  ]]
Wall time: 3min 48s


In [124]:
centroids.shape

(5, 9328)

In [125]:
centroids

array([[0.2       , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.16666667, 0.        ,
        0.        ],
       [0.00296736, 0.        , 0.19287834, ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.        , 0.        , ..., 0.14285714, 0.07142857,
        0.42857143],
       [0.0018315 , 0.002442  , 0.004884  , ..., 0.        , 0.        ,
        0.        ]])

In [126]:
clusterAssment[:10]

matrix([[4.        , 0.78477557],
        [4.        , 0.72171818],
        [4.        , 0.71806336],
        [4.        , 0.73483166],
        [4.        , 0.63579767],
        [4.        , 0.65164049],
        [4.        , 0.45891466],
        [2.        , 0.52808895],
        [4.        , 0.7550924 ],
        [4.        , 0.60151329]])

In [127]:
term_list = pd.Series(terms[0], name='Term')

term_list.shape

(9328,)

In [128]:
centroid = pd.Series(centroids[0], name='Centroid_value')
centroid

0       0.2
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9323    0.0
9324    0.0
9325    0.0
9326    0.0
9327    0.0
Name: Centroid_value, Length: 9328, dtype: float64

In [129]:
clust = pd.concat([term_list, centroid],axis=1)
clust

,Term,Centroid_value
0,aa,0.2
1,aargh,0.0
2,aaron,0.0
3,aaronc,0.0
4,ab,0.0
...,...,...
9323,zubov,0.0
9324,zv,0.0
9325,zw,0.0
9326,zx,0.0


In [130]:
clusters = pd.DataFrame(clusterAssment)
print(clusters.shape)
print(clusters.index[clusters.iloc[:,0]==0])
cl_index = np.array(clusters.index[clusters.iloc[:,0]==0])
cl_index

(2000, 2)
Int64Index([316, 787, 1124, 1515, 1658], dtype='int64')


array([ 316,  787, 1124, 1515, 1658], dtype=int64)

In [92]:
clust_df = clust.iloc[cl_index]
        #.head()
clust_df.sort_values(by='Centroid_value', axis=0, ascending=False, inplace=True)
clust_df.head(5)

<ipython-input-92-d49903f3e538>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clust_df.sort_values(by='Centroid_value', axis=0, ascending=False, inplace=True)


,Term,Centroid_value
1894,cup,0.336634
977,bruin,0.222772
887,boston,0.220297
1451,coach,0.190594
1977,david,0.118812


In [145]:
def top_terms(N, centroids, clusters,range_val):
    term_list = pd.Series(terms[0], name='Term')
    clusters = pd.DataFrame(clusters)
    for i in range(range_val):
        cl_index = np.array(clusters.index[clusters.iloc[:,0]==i])
        centroid = pd.Series(centroids[i], name='Centroid_value')
        clust = pd.concat([term_list, centroid],axis=1)
        clust_docs = len(cl_index)
        print('\nCluster: ',i,' results')
        print('Number of docuemnts in cluster',str(clust_docs))
        print("\tword\tfrequency\tDF\tPercentOfDocs")
        clust_df = clust.iloc[cl_index]
        #.head()
        clust_df.sort_values(by='Centroid_value', axis=0, ascending=False, inplace=True)
        top_n = clust_df.head(N)
        for t in top_n.itertuples():
            doc_freq = t.Centroid_value * clust_docs
            mean_freq = np.mean(t.Centroid_value)
            PercentOfDocs = t.Centroid_value * 100
            print(term_list[term_list == t.Term].index[0],'\t',t.Term,'\t',mean_freq,'\t',doc_freq,'\t',PercentOfDocs)
              #print("%s\t%10s\t\t%.0f\t%.6f" %(term_list[term_list == t.Term].index[0],t.Term, doc_freq, PercentOfDocs))

In [146]:
%%time
range_val = 5
top_terms(10, centroids, clusterAssment,range_val)


Cluster:  0  results
Number of docuemnts in cluster 5
	word	frequency	DF	PercentOfDocs
316 	 annual 	 3.2 	 16.0 	 320.0
787 	 birth 	 0.0 	 0.0 	 0.0
1124 	 caralvautotrolcom 	 0.0 	 0.0 	 0.0
1515 	 commision 	 0.0 	 0.0 	 0.0
1658 	 content 	 0.0 	 0.0 	 0.0

Cluster:  1  results
Number of docuemnts in cluster 6
	word	frequency	DF	PercentOfDocs
434 	 ashton 	 0.0 	 0.0 	 0.0
482 	 atlanta 	 0.0 	 0.0 	 0.0
843 	 bo 	 0.0 	 0.0 	 0.0
968 	 broken 	 0.0 	 0.0 	 0.0
1925 	 cypherpunk 	 0.0 	 0.0 	 0.0
1990 	 dcsedacuk 	 0.0 	 0.0 	 0.0

Cluster:  2  results
Number of docuemnts in cluster 337
	word	frequency	DF	PercentOfDocs
1333 	 christ 	 0.7388724035608308 	 249.0 	 73.88724035608308
1176 	 cathol 	 0.39465875370919884 	 133.0 	 39.46587537091988
31 	 accept 	 0.3827893175074184 	 129.0 	 38.27893175074184
604 	 baptism 	 0.22255192878338279 	 75.0 	 22.255192878338278
131 	 against 	 0.20178041543026706 	 68.0 	 20.178041543026705
1524 	 commun 	 0.17210682492581603 	 58.0 	 17.210

<ipython-input-145-99eae0a1cde9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clust_df.sort_values(by='Centroid_value', axis=0, ascending=False, inplace=True)


### Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [152]:
vs_target_train

array([4, 0, 4, ..., 2, 0, 1])

In [155]:
from sklearn.metrics import completeness_score, homogeneity_score

#NOTE: this generates a random score every time you run the cluster
print("completeness_score = ",completeness_score(vs_target_train,np.ravel(clusterAssment.T[0])))
print("homogeneity_score = ",homogeneity_score(vs_target_train,np.ravel(clusterAssment.T[0])))

completeness_score =  0.684620979720857
homogeneity_score =  0.22579135963730823


### Finally, using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and cluster centroids. For each test document show the predicted class label as well as Cosine similarity to the corresponding cluster.

In [219]:

results = []

for doc in vs_test:
    dic = {}
    sim = []
    cluster = 0
    for centroid in centroids:
        cluster += 1
        val = np.ravel(calculateCosineSimilarityDistance(centroid,doc))[0]
        #print("val ",val)
        centroid_doc_sim = 1 - val #Need to subtract from 1 as our function returns distance and not similarity
        #print("centroid_doc_sim ",centroid_doc_sim)
        sim.append(centroid_doc_sim) 
        dic['Cosine Similarity to Cluster '+str(cluster)] = str(np.ravel(centroid_doc_sim)[0])
    dic['Predicted Cluster'] = sim.index(max(sim))+1
    results.append(dic)

test_cluster = pd.DataFrame(results)

In [220]:
test_cluster

,Cosine Similarity to Cluster 1,Cosine Similarity to Cluster 2,Cosine Similarity to Cluster 3,Cosine Similarity to Cluster 4,Cosine Similarity to Cluster 5,Predicted Cluster
0,0.010368102191173012,2.343730792664367e-05,0.20697149600801568,0.00887803631030848,0.16697281596113767,3
1,0.024122819646888116,6.536352890607056e-06,0.19230997935094207,0.007018954756392248,0.16871804941603252,3
2,0.014172900315532977,3.100464408734016e-05,0.11021237054146171,0.037074650014280564,0.1633919460161013,5
3,0.022366955566112234,2.7167084271306763e-05,0.21383908807818708,0.017429586794587726,0.3709520576683851,5
4,0.012403444013234122,1.8186966430633333e-05,0.06211178414194618,0.013964615920429635,0.161929696659889,5
...,...,...,...,...,...,...
495,0.01085124986077024,1.3208177853107905e-05,0.1500705859337812,0.030599019639553138,0.3220987774599666,5
496,0.027802285074474575,2.8500672307196773e-05,0.08935738126955872,0.017955983859736846,0.19274233344094494,5
497,0.012438027307374089,1.5745193049920836e-05,0.16839844671236115,0.008842776350630577,0.1126000649175869,3
498,0.014380594559384341,3.0475902494164764e-05,0.24217844182931147,0.013104285086659617,0.2955659667568775,5
